In [2]:
import glob
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import time
from tqdm import tqdm
# from load_data import extract_tar, readJSONEntries
# from data_preparation import *
# from sentiment_analysis import bert_sentiment_pred
import warnings
warnings.filterwarnings('ignore')

In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.3
tf.compat.v1.Session(config=config)

In [3]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 

['/device:CPU:0']


In [ ]:
tokenizer = BertTokenizer.from_pretrained("C:/Users/Ale/Desktop/Tesi/biobert_v1.1_pubmed/biobert_v1.1_pubmed", from_pt = True)

model = TFBertForSequenceClassification.from_pretrained("C:/Users/Ale/Desktop/Tesi/finetuned_model/")

In [6]:
df_com_a = pd.read_csv("./Data_asthma/Official/df_com_off_clean.csv", parse_dates = ["time"])
# df_com_l = pd.read_csv("./Data_lungs/df_com_clean.csv", parse_dates = ["time"])

In [7]:
df_com_a['sentiment'] = 0
# df_com_l['sentiment'] = 0      
for i, text in enumerate(tqdm(df['text'])):
    tf_batch = tokenizer(text, max_length=128, padding=True, truncation=True, return_tensors='tf')
    output = model(tf_batch)
    tf_predictions = tf.nn.softmax(output[0], axis=-1)
    label = tf.argmax(tf_predictions, axis=1)
    label = label.numpy()
    df_com_a['sentiment'].iloc[i] = label
df_com_a.to_csv(f"./Data_asthma/Official/df_com_off_clean.csv", index = False)
#     df_com_l.to_csv(f"./Data_lungs/df_com_clean.csv", index = False)

100%|██████████████████████████████████████████████████████████████████████████████| 2500/2500 [20:20<00:00,  2.05it/s]


In [1]:
import multiprocessing
import time
from transformers import BertTokenizer, TFBertForSequenceClassification


def bert_sentiment_pred(text, model = None):
    name = multiprocessing.current_process().name
    
    print(name, 'Starting')
    if not model:
        tokenizer = BertTokenizer.from_pretrained("dmis-lab/biobert-v1.1", from_pt = True)
        model = TFBertForSequenceClassification.from_pretrained("./tesi_pretrained_SA_bio_bert/")
    
    tf_batch = tokenizer(text, max_length=128, padding=True, truncation=True, return_tensors='tf')
    result = model(tf_batch)
    
    print(name, 'Exiting')
    return result

In [61]:
df_com_a.shape

(87018, 10)